In [4]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.get_dataframe()


# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

In [5]:
df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df
print(len(df.CUSTOMER.unique()))

657500


In [0]:
# find all customer_names that have other customer names that start with this customer name

idx = 0
unique_customer_names = df['CUSTOMER'].unique()
full_set_n = len(unique_customer_names)

list_customers_ = []
list_potential_matches_ = []

report_every_n = 10

for n in unique_customer_names:
    idx+=1
    df_f = df[(df['CUSTOMER'].str.startswith(n, na=False))&(df['CUSTOMER']!=n)]
    match_list = df_f['CUSTOMER'].unique()
    if len(match_list)>0:
        list_customers_.append(n)
        list_potential_matches_.append(match_list)

    if (idx % report_every_n == 0):
        print(idx, "iterations", len(list_customers_), "with potential matche(s)", round(idx/full_set_n,2), "% complete")
        
    if idx>100:
        break;
        
print(len(list_customers_))

10 iterations 0 with potential matche(s) 0.0 % complete
20 iterations 1 with potential matche(s) 0.0 % complete


In [0]:
ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df # For this sample code, simply copy input to output


# Write recipe outputs
ACCOUNTS_WITH_POTENTIAL_DUPLICATES = dataiku.Dataset("ACCOUNTS_WITH_POTENTIAL_DUPLICATES")
ACCOUNTS_WITH_POTENTIAL_DUPLICATES.write_with_schema(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df)